In [15]:
# -*- coding: utf-8 -*-
"""
Only tested in Python 3.
You may need to install the 'requests' Python3 module.
Be sure to fill in your username, password, org name and email before running
"""

from os import path
import requests
from requests.auth import HTTPBasicAuth

# account details
USERNAME = 'melissa'
PASSWORD = 'WattTime4Metis'
EMAIL = 'melissa@doloresparkpiano.com'
ORG = 'StudentsInc'

# request details
BA = 'CAISO_NORTH'  # identify grid region

# starttime and endtime are optional, if ommited will return the latest value
START = '2021-09-08T19:00:00-0000'  # UTC offset of 0 (PDT is -7, PST -8)
END = '2021-09-08T19:45:00-0000'


def register(username, password, email, org):
    url = 'https://api2.watttime.org/register'
    params = {'username': username,
              'password': password,
              'email': email,
              'org': org}
    rsp = requests.post(url, json=params)
    print(rsp.text)


def login(username, password):
    url = 'https://api2.watttime.org/login'
    try:
        rsp = requests.get(url, auth=HTTPBasicAuth(username, password))
    except BaseException as e:
        print('There was an error making your login request: {}'.format(e))
        return None

    try:
        token = rsp.json()['token']
    except BaseException:
        print('There was an error logging in. The message returned from the '
              'api is {}'.format(rsp.text))
        return None

    return token


def data(token, ba, starttime, endtime):
    url = 'https://api2.watttime.org/data'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba, 'starttime': starttime, 'endtime': endtime}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def index(token, ba):
    url = 'https://api2.watttime.org/index'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def forecast(token, ba, starttime=None, endtime=None):
    url = 'https://api2.watttime.org/forecast'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    if starttime:
        params.update({'starttime': starttime, 'endtime': endtime})

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def historical(token, ba):
    url = 'https://api2.watttime.org/historical'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    rsp = requests.get(url, headers=headers, params=params)
    cur_dir = path.dirname(path.realpath('my_project'))
    file_path = path.join(cur_dir, '{}_historical.zip'.format(ba))
    with open(file_path, 'wb') as fp:
        fp.write(rsp.content)

    print('Wrote historical data for {} to {}'.format(ba, file_path))

In [2]:
# Only register once!!
# register(USERNAME, PASSWORD, EMAIL, ORG)

{"user":"melissa","ok":"User created"}


In [21]:
token = login(USERNAME, PASSWORD)
if not token:
    print('You will need to fix your login credentials (username and password '
          'at the start of this file) before you can query other endpoints. '
          'Make sure that you have registered at least once by uncommenting '
          'the register(username, password, email, org) line near the bottom '
          'of this file.')
    exit()

In [22]:
realtime_index = index(token, BA)
print(realtime_index)

{'freq': '300', 'ba': 'CAISO_NORTH', 'percent': '94', 'moer': '1007', 'point_time': '2021-09-13T22:25:00Z'}


In [17]:
type(realtime_index)

dict

In [6]:
print('Please note: the following endpoints require a WattTime subscription')
historical_moer = data(token, BA, START, END)
print(historical_moer)

Please note: the following endpoints require a WattTime subscription
[{'point_time': '2021-09-08T19:45:00.000Z', 'value': 950.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'point_time': '2021-09-08T19:40:00.000Z', 'value': 950.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'point_time': '2021-09-08T19:35:00.000Z', 'value': 950.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'point_time': '2021-09-08T19:30:00.000Z', 'value': 949.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'point_time': '2021-09-08T19:25:00.000Z', 'value': 948.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'point_time': '2021-09-08T19:20:00.000Z', 'value': 947.0, 'frequency': 300, 'market': 'RTM', 'ba': 'CAISO_NORTH', 'datatype': 'MOER', 'version': '3.0'}, {'poin

In [19]:
type(historical_moer[0])

dict

In [7]:
forecast_moer = forecast(token, BA)
print(forecast_moer)

{'generated_at': '2021-09-09T02:55:00+00:00', 'forecast': [{'point_time': '2021-09-09T03:00:00+00:00', 'value': 917.4613204100873, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:05:00+00:00', 'value': 918.5611769152166, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:10:00+00:00', 'value': 923.2322511140461, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:15:00+00:00', 'value': 929.6538042438821, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:20:00+00:00', 'value': 935.3723206575347, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:25:00+00:00', 'value': 928.8773884259017, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:30:00+00:00', 'value': 931.8671770686624, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'}, {'point_time': '2021-09-09T03:35:00+00:00', 'value': 936.7406135709188, 'version': '3.0-1.0.0', 'ba': 'CAISO_NORTH'

In [8]:
forecast_moer = forecast(token, BA, START, END)
print(forecast_moer)

[{'forecast': [{'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:05:00+00:00', 'value': 954.4333143775001, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:10:00+00:00', 'value': 953.7777518366564, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:15:00+00:00', 'value': 955.7705757236876, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:20:00+00:00', 'value': 953.5611710078884, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:25:00+00:00', 'value': 956.1954224650304, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:30:00+00:00', 'value': 954.2541086264886, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:35:00+00:00', 'value': 954.0350510202496, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021-09-08T19:40:00+00:00', 'value': 957.4329575752339, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2021

In [16]:
historical(token, BA)  # Writes zip file to current directory

Wrote historical data for CAISO_NORTH to /home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical.zip


In [51]:
START = '2019-12-01T11:55:00-0000'
END = '2019-12-01T12:00:00-0000'
forecast_moer = forecast(token, BA, START, END)
print(forecast_moer)

{'error': 'Invalid query parameters', 'message': 'Could not find any forecasts in the specified start-end range for this BA'}


In [53]:
# historical forecast data only available from 2020 on
START = '2019-12-31T23:55:00-0000'
END = '2020-01-01T00:00:00-0000'
forecast_moer = forecast(token, BA, START, END)
print(forecast_moer)

[{'forecast': [{'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:05:00+00:00', 'value': 999.2366698289579, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:10:00+00:00', 'value': 1005.9866435040786, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:15:00+00:00', 'value': 1014.2005580867959, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:20:00+00:00', 'value': 1005.8979698456591, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:25:00+00:00', 'value': 1005.5158557003882, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:30:00+00:00', 'value': 1017.4024840782631, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:35:00+00:00', 'value': 1038.768062873276, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:40:00+00:00', 'value': 1040.1364897187116, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time':

In [54]:
len(forecast_moer[0]['forecast'])

288

In [55]:
print(forecast_moer[0]['forecast'][:5])

[{'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:05:00+00:00', 'value': 999.2366698289579, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:10:00+00:00', 'value': 1005.9866435040786, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:15:00+00:00', 'value': 1014.2005580867959, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:20:00+00:00', 'value': 1005.8979698456591, 'version': '3.0-1.0.0'}, {'ba': 'CAISO_NORTH', 'point_time': '2020-01-01T00:25:00+00:00', 'value': 1005.5158557003882, 'version': '3.0-1.0.0'}]
